In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import time
from datetime import datetime
#from Wisard import Wisard
import wisardpkg as wp
import math
import sys
import pickle

In [2]:
params = {}

#file = open("compact_Health_Sports_50.pickle", "rb")
#words_data = pickle.load(file)
words_data = None

In [3]:
# The preprocessing part is partialy based on the following "kernel" on Kaggle:
# https://www.kaggle.com/jgoldberg/donorschoose-eda-text-classification/notebook

def preprocess(training_dataframe, resources_dataframe):
    #print(training_dataframe.shape)
    #print(resources_dataframe.shape)
    
    #
    total_price = resources_dataframe.quantity * resources_dataframe.price
    resources_dataframe["total_price"] = total_price
    
    # dropping irrelevant columns
    resources_dataframe = resources_dataframe.drop(["description", "price"], axis=1)
    training_dataframe = training_dataframe.drop(["teacher_id"], axis=1)
    
    # grouping resources data by id
    grouped_resources_dataframe = resources_dataframe.groupby("id", as_index=False, sort=False).sum()
    grouped_resources_dataframe
    
    # merging the two dataframes
    cleaned_df = pd.merge(training_dataframe, grouped_resources_dataframe, how="inner", on=["id"])
    
    # splitting project categories
    
    #cleaned_df[['category_1','category_2']] = cleaned_df['project_subject_categories'].str.replace(", Care & Hunger", "").str.split(', ', 3, expand=True)
    ##print(cleaned_df['project_subject_categories'].str.replace(", Care & Hunger", "").str.split(', ', 3, expand=True))
    
    #cleaned_df[['subcategory_1','subcategory_2']] = cleaned_df['project_subject_subcategories'].str.replace(", Care & Hunger", "").str.split(', ', 3, expand=True)
    
    ##cleaned_df["category_1"] = cleaned_df["category_1"].fillna("Not Informed")
    #cleaned_df["category_2"] = cleaned_df["category_2"].fillna("Not Informed")
    #cleaned_df["subcategory_2"] = cleaned_df["subcategory_2"].fillna("Not Informed")
    
    cleaned_df["total_price_category"] = pd.cut(
        cleaned_df["total_price"], 
        bins=[0,100,250,500,1000,16000], 
        labels=["0-100","101-250","251-500","501-1000",">1000"]
    )
    
    cleaned_df["n_previous_projects"] = pd.cut(
        cleaned_df["teacher_number_of_previously_posted_projects"],
        bins=[-1,1,5,10,25,50,500],
        labels=['0-1','2-5','6-10','11-25','26-50','51+']
    )
    
    cleaned_df["project_submitted_datetime"] = pd.to_datetime(cleaned_df['project_submitted_datetime'])
    cleaned_df["month"] = cleaned_df['project_submitted_datetime'].dt.month
    cleaned_df["quarter"] = cleaned_df['project_submitted_datetime'].dt.quarter
    
    cleaned_df["teacher_prefix"] = cleaned_df["teacher_prefix"].fillna("unknown")
    
    cleaned_df["project_essay_1"] = cleaned_df["project_essay_1"].fillna("")
    cleaned_df["project_essay_2"] = cleaned_df["project_essay_2"].fillna("")
    cleaned_df["project_essay_3"] = cleaned_df["project_essay_3"].fillna("")
    cleaned_df["project_essay_4"] = cleaned_df["project_essay_4"].fillna("")
    
    #cleaned_df["merged_essays"] = cleaned_df['project_title'].astype(str) + " " + cleaned_df['project_essay_1'].astype(str) + " " + cleaned_df['project_essay_2'].astype(str) + " " + cleaned_df['project_essay_3'].astype(str) + " " + cleaned_df['project_essay_4'].astype(str)
    cleaned_df["merged_essays"] = cleaned_df['project_essay_1'].astype(str) + " " + cleaned_df['project_essay_2'].astype(str) + " " + cleaned_df['project_essay_3'].astype(str) + " " + cleaned_df['project_essay_4'].astype(str)
    
    # dropping more columns
    cleaned_df = cleaned_df.drop([
        "project_submitted_datetime", 
        "project_essay_1", 
        "project_essay_2", 
        "project_essay_3", 
        "project_essay_4",
        "quantity",
        "total_price",
        "teacher_number_of_previously_posted_projects"], 
        axis=1
    )
    
    #cleaned_df = pd.merge(cleaned_df, words_data, how="inner", on=["id"])
    
    print("Dim.:", cleaned_df.shape)
    
    return cleaned_df


def get_bits_string(value_order, encoding, min_lenght, length, thershold = None):
    if (value_order <= 0) or (value_order > length):
        print("Value order should be 0 < value_order <= length.")
        #sys.exit(1)
        exit()
    
    string = ""
    
    # 2 = 0001, if length = 4
    if encoding == "standard": 
        if (value_order - 1) > math.pow(2, length):
            print("It is not possible to represent " + str(value_order) + " different values with " + str(length) + " bits.")
        #sys.exit(1)
        exit()
        
        string = ("{0:0" + str(length) + "b}").format(value_order - 1)
        
    # 2 = 1100, if length = 4
    elif encoding == "thermometer": 
        value_size = math.floor(length / min_lenght) 
        string = "1" * (value_order * value_size) + "0" * (length - (value_order * value_size))
    
    # 2 = 0100, if length = 4
    elif encoding == "onehot":
        string = "0" * length
        string = list(string)
        string[value_order - 1] = "1"
        string = "".join(string)
        
    elif encoding == "threshold":
        if threshold != None:
            if value_order > threshold:
                string = "1"
            else:
                string = "0"
        else:
            print("Threshold not provided.")
            #sys.exit(1)
            exit()
    
    else:
        print("Invalid encoding: " + str(encoding) + ".\nValid encodings are \"standard\", \"thermometer\", \"onehot\" and \"threshold.\".")
        #sys.exit(1)
        exit()
        
    return string

def choose_field_size(minimum_size, expected_size):
    if minimum_size > expected_size:
        return minimum_size
    else:
        return expected_size
    

def encode_data(value_order, encoding, minimum_length, desired_length, threshold):
    return get_bits_string(
        value_order, 
        encoding, 
        minimum_length,
        choose_field_size(
            minimum_length, 
            desired_length
        ), 
        threshold
    )
    

def convert_to_bits_string(dataframe, params):
    print("Converting dataframe of shape ", dataframe.shape, " to a list of binary values.")
    
    #project_grade_category_mapping = { # 4
    #    'Grades PreK-2':  get_bits_string(1, params["encoding"], choose_field_size(4, params["field_size"]), params["encoding_threshold"]),
    #    'Grades 3-5':     get_bits_string(2, params["encoding"], choose_field_size(4, params["field_size"]), params["encoding_threshold"]), 
    #    'Grades 6-8':     get_bits_string(3, params["encoding"], choose_field_size(4, params["field_size"]), params["encoding_threshold"]), 
    #    'Grades 9-12':    get_bits_string(4, params["encoding"], choose_field_size(4, params["field_size"]), params["encoding_threshold"])
    #}
    print(1, params["encoding"], 4, params["field_size"], params["encoding_threshold"])
    project_grade_category_mapping = { # 4
        'Grades PreK-2':  encode_data(1, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
        'Grades 3-5':     encode_data(2, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
        'Grades 6-8':     encode_data(3, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
        'Grades 9-12':    encode_data(4, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
    }

    #teacher_prefix_mapping = { # 6
    #    'Ms.':     get_bits_string(1, params["encoding"], choose_field_size(6, params["field_size"]), params["encoding_threshold"]), 
    #    'Mrs.':    get_bits_string(2, params["encoding"], choose_field_size(6, params["field_size"]), params["encoding_threshold"]), 
    #    'Mr.':     get_bits_string(3, params["encoding"], choose_field_size(6, params["field_size"]), params["encoding_threshold"]), 
    #    'Teacher': get_bits_string(4, params["encoding"], choose_field_size(6, params["field_size"]), params["encoding_threshold"]), 
    #    'Dr.':     get_bits_string(5, params["encoding"], choose_field_size(6, params["field_size"]), params["encoding_threshold"]), 
    #    'unknown': get_bits_string(6, params["encoding"], choose_field_size(6, params["field_size"]), params["encoding_threshold"])
    #}
    
    teacher_prefix_mapping = { # 6
        'Ms.':     encode_data(1, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        'Mrs.':    encode_data(2, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        'Mr.':     encode_data(3, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        'Teacher': encode_data(4, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        'Dr.':     encode_data(5, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        'unknown': encode_data(6, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
    }

    n_previous_projects_mapping = { # 6
        '0-1':     encode_data(1, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        '2-5':     encode_data(2, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        '6-10':    encode_data(3, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        '11-25':   encode_data(4, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        '26-50':   encode_data(5, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        '51+':     encode_data(6, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
    }

    total_price_category_mapping = { # 6
        "0-100":     encode_data(1, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        "101-250":   encode_data(2, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        "251-500":   encode_data(3, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        "501-1000":  encode_data(4, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
        ">1000":     encode_data(5, params["encoding"], 6, params["field_size"], params["encoding_threshold"]),
    }
    
    #month_mapping = { # 12
    #    "1":  get_bits_string(1, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "2":  get_bits_string(2, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "3":  get_bits_string(3, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "4":  get_bits_string(4, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "5":  get_bits_string(5, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "6":  get_bits_string(6, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "7":  get_bits_string(7, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "8":  get_bits_string(8, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "9":  get_bits_string(9, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "10": get_bits_string(10, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "11": get_bits_string(11, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"]),
    #    "12": get_bits_string(12, params["encoding"], choose_field_size(12, params["field_size"]), params["encoding_threshold"])
    #}
    
    month_mapping = { # 12
        "1":  encode_data(1, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "2":  encode_data(2, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "3":  encode_data(3, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "4":  encode_data(4, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "5":  encode_data(5, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "6":  encode_data(6, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "7":  encode_data(7, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "8":  encode_data(8, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "9":  encode_data(9, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "10": encode_data(10, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "11": encode_data(11, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
        "12": encode_data(12, params["encoding"], 12, params["field_size"], params["encoding_threshold"]),
    }
    
    quarter_mapping = { # 4
        "1": encode_data(1, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
        "2": encode_data(2, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
        "3": encode_data(3, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
        "4": encode_data(4, params["encoding"], 4, params["field_size"], params["encoding_threshold"]),
    }
    
    category_mapping = { # 10
        "Not Informed":          encode_data(1, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Applied Learning":      encode_data(2, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Health & Sports":       encode_data(3, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "History & Civics":      encode_data(4, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Literacy & Language":   encode_data(5, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Math & Science":        encode_data(6, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Music & The Arts":      encode_data(7, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Special Needs":         encode_data(8, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Warmth":                encode_data(9, params["encoding"], 10, params["field_size"], params["encoding_threshold"]),
        "Care & Hunger":         encode_data(9, params["encoding"], 10, params["field_size"], params["encoding_threshold"]), # Equals to warmth, because they are the same thing
    }
    
    subcategory_mapping = { # 30
        "Not Informed":          encode_data(1, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Literacy":              encode_data(2, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Performing Arts":       encode_data(3, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Applied Sciences":      encode_data(4, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Health & Wellness":     encode_data(5, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Character Education":   encode_data(6, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Early Development":     encode_data(7, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Mathematics":           encode_data(8, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Literature & Writing":  encode_data(9, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Special Needs":         encode_data(10, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "ESL":                   encode_data(11, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Health & Life Science": encode_data(12, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "College & Career Prep": encode_data(13, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Environmental Science": encode_data(14, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Other":                 encode_data(15, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Music":                 encode_data(16, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Visual Arts":           encode_data(17, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "History & Geography":   encode_data(18, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Gym & Fitness":         encode_data(19, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Warmth":                encode_data(20, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Extracurricular":       encode_data(21, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Team Sports":           encode_data(22, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Social Sciences":       encode_data(23, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Foreign Languages":     encode_data(24, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Parent Involvement":    encode_data(25, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Nutrition Education":   encode_data(26, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Community Service":     encode_data(27, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Financial Literacy":    encode_data(28, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Civics & Government":   encode_data(29, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        "Economics":             encode_data(30, params["encoding"], 30, params["field_size"], params["encoding_threshold"]),
        
    }

    school_state_mapping = { # 52 (51 possible values)
        'NV': encode_data(1, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'GA': encode_data(2, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'UT': encode_data(3, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'NC': encode_data(4, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'CA': encode_data(5, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'DE': encode_data(6, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'MO': encode_data(7, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'SC': encode_data(8, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'IN': encode_data(9, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'IL': encode_data(10, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'VA': encode_data(11, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'PA': encode_data(12, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'NY': encode_data(13, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'FL': encode_data(14, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'NJ': encode_data(15, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'TX': encode_data(16, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'LA': encode_data(17, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'ID': encode_data(18, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'OH': encode_data(19, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'OR': encode_data(20, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'MD': encode_data(21, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'WA': encode_data(22, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'MA': encode_data(23, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'KY': encode_data(24, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'AZ': encode_data(25, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'MI': encode_data(26, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'CT': encode_data(27, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'AR': encode_data(28, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'WV': encode_data(29, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'NM': encode_data(30, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'WI': encode_data(31, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'MN': encode_data(32, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'OK': encode_data(33, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'AL': encode_data(34, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'TN': encode_data(35, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'IA': encode_data(36, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'KS': encode_data(37, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'CO': encode_data(38, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'DC': encode_data(39, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'WY': encode_data(40, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'NH': encode_data(41, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'HI': encode_data(42, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'SD': encode_data(43, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'MT': encode_data(44, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'MS': encode_data(45, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'RI': encode_data(46, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'VT': encode_data(47, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'ME': encode_data(48, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'NE': encode_data(49, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'AK': encode_data(50, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
        'ND': encode_data(51, params["encoding"], 52, params["field_size"], params["encoding_threshold"]),
    }
    
    combined_input_and_expected_output = []
    input_list = []
    expected_output_list = []
    
    n = 0
    num_bits = 0
    for index, row in dataframe.iterrows():
        #print(row)
        #if n >= 10:
        #    break
        #n = n + 1
        
        # total 128 bits
        bits_string = ""
        bits_string = project_grade_category_mapping[row["project_grade_category"]]
        bits_string = bits_string + teacher_prefix_mapping[row["teacher_prefix"]]
        bits_string = bits_string + n_previous_projects_mapping[row["n_previous_projects"]]
        bits_string = bits_string + total_price_category_mapping[row["total_price_category"]]
        
        bits_string = bits_string + month_mapping[str(row["month"])]
        bits_string = bits_string + quarter_mapping[str(row["quarter"])]
        #bits_string = bits_string + category_mapping[row["category_1"]]
        #bits_string = bits_string + category_mapping[row["category_2"]] # perhaps it is possible to ignore this one
        
        #bits_string = bits_string + subcategory_mapping[row["subcategory_1"]]
        #bits_string = bits_string + subcategory_mapping[row["subcategory_2"]]
        bits_string = bits_string + school_state_mapping[row["school_state"]]
        
        words_bits_columns = [col for col in dataframe.columns if col.startswith("col")]
        
        words_bits_string = ""
        
        for column in words_bits_columns:
            words_bits_string = words_bits_string + str(row[column])
            
        bits_string = bits_string + words_bits_string
        
        num_bits = len(bits_string)
        
        bit_int_list = [int(c) for c in bits_string]
        expected_output = str(row["project_is_approved"])
        
        input_list.append(bit_int_list)
        expected_output_list.append(expected_output)
        
        combined_input_and_expected_output.append([bit_int_list, expected_output])
        
    print("Number of bits in the input: ", num_bits)
        
    return input_list, expected_output_list, combined_input_and_expected_output

In [4]:
def loadData():
    # load data
    train_file_path = 'train.csv'
    # Test data file not considered because it doesn't contain the classes of the entries
    #test_file_path = 'test.csv'
    resources_file_path = 'resources.csv'

    # Read data and store in DataFrame
    train_data = pd.read_csv(train_file_path, sep=',')
    #test_data = pd.read_csv(test_file_path, sep=',')
    resources_data = pd.read_csv(resources_file_path, sep=',')
    
    return train_data, resources_data

# splitting the training dataset into training and test, because the official test dataset
# doesn't have the entries' classification, requiring validation with Kaggle's website
def splitData(train_data, resources_data, training_set_total_aproved, training_set_total_reproved):
    
    print("Total data: ", len(train_data))
    print("Total aproved: ", train_data["project_is_approved"].sum())
    print("Total reproved: ", len(train_data) - train_data["project_is_approved"].sum())
    print("Percent aproved: ", float(train_data["project_is_approved"].sum()) / float(len(train_data)))
    print("Percent reproved: ", 1.0 - (float(train_data["project_is_approved"].sum()) / float(len(train_data))), "\n")

    #train = train_data.sample(n=10000,random_state=200)
    train = train_data.sample(frac=1,random_state=200)
    print("Distribution over a random sample of 182080 observations used to get the observations to train the classifier: ",
          float(train["project_is_approved"].sum()) / float(len(train["project_is_approved"])))
    print("Total aproved in that sample: ", train["project_is_approved"].sum(), "\n")
    
    aproved = train[train["project_is_approved"] == 1][:training_set_total_aproved]
    reproved = train[train["project_is_approved"] == 0][:training_set_total_reproved]

    training_set = pd.concat([aproved, reproved])
    training_set = training_set.sample(frac=1, random_state=200)
    test_set = train_data.drop(training_set.index)

    print("Total training data: ", len(training_set))
    print("Total aproved: ", training_set["project_is_approved"].sum())
    print("Total reproved: ", len(training_set) - training_set["project_is_approved"].sum())
    print("Percent aproved: ", float(training_set["project_is_approved"].sum()) / float(len(training_set)))
    print("Percent reproved: ", 1.0 - (float(training_set["project_is_approved"].sum()) / float(len(training_set))), "\n")

    print("Total test data: ", len(test_set))
    print("Total aproved: ", test_set["project_is_approved"].sum())
    print("Total reproved: ", len(test_set) - test_set["project_is_approved"].sum())
    print("Percent aproved: ", float(test_set["project_is_approved"].sum()) / float(len(test_set)))
    print("Percent reproved: ", 1.0 - (float(test_set["project_is_approved"].sum()) / float(len(test_set))), "\n")

    print("Training set + test set: ", len(training_set) + len(test_set))

    return training_set, test_set


def getData(training_set_total_aproved, training_set_total_reproved, params):
    train_data, resources_data = loadData()
    train_data = pd.merge(train_data, params["category_text_data"], how="inner", on=["id"])
    print("train_data shape", train_data.shape)
    training_set, test_set = splitData(train_data, resources_data, training_set_total_aproved, training_set_total_reproved)
    
    training_df = preprocess(training_set, resources_data)
    test_df = preprocess(test_set, resources_data)
    
    training_input, expected_output, training_combined = convert_to_bits_string(training_df, params)
    test_input, test_expected_output, test_combined = convert_to_bits_string(test_df, params)
    
    return training_input, expected_output, training_combined, test_input, test_expected_output, test_combined, training_set, test_set

In [5]:
# Trains using a WiSARD classifier
# Using personal implementation, without bleaching
def train(algorithm, training_input, expected_output, params, tuple_size = 2, bleaching = False, seed = 0):
    #wann = Wisard(tuple_size, 3546, bleaching)
    #wann.train(training_input, expected_output)
    
    wann = None
    
    if algorithm == "wisard":
        print("Algorithm: WiSARD.")
        wann = wp.Wisard(tuple_size, bleachingActivated=bleaching, ignoreZero=False, verbose=True)
        wann.train(training_input,expected_output)
    elif algorithm == "cluswisard":
        print("Algorithm: ClusWiSARD.")
        wann = wp.ClusWisard(tuple_size, params["min_score"], params["threshold"], params["discriminator_limit"])
        wann.train(training_input,expected_output)
    
    
    return wann

In [6]:
#Evaluates Guilherme's wisard implementation
def evaluate_performance(wann, test_data_combined, current_params):
    #print("Number of observations: ", test_data_combined)
    correct_predictions = 0
    wrong_predictions = 0
    zeros_predicted = 0
    ones_predicted = 0
    zeros_correct = 0
    ones_correct = 0
    zeros_wrong = 0
    ones_wrong = 0
    total_ties = 0
    ones_total_ties = 0
    zeros_total_ties = 0
    avg_time = time.time()
    
    for combined in test_data_combined:
        #prediction, tie = wann.classify([combined[0]], returnActivationDegree=True, returnConfidence=True)
        prediction = wann.classify([combined[0]])#, searchBestConfidence = False)#, returnActivationDegree=True, returnConfidence=True)
        #print(prediction)
        prediction = prediction[0]#["class"]
        tie = 0
        
        if prediction == "0":
            #print("Prediction: ", prediction[0], combined)
            zeros_predicted = zeros_predicted + 1
        elif prediction == "1":
            ones_predicted = ones_predicted + 1
        #print(prediction)
        
        expected = combined[1]
        #print(prediction, expected)
        if prediction == expected:
            #print("Correct!")
            correct_predictions = correct_predictions + 1
            
            if prediction == "0":
                zeros_correct = zeros_correct + 1
            elif prediction == "1":
                ones_correct = ones_correct + 1
        else:
            wrong_predictions = wrong_predictions + 1
            
            if prediction == "0":
                zeros_wrong = zeros_wrong + 1
            elif prediction == "1":
                ones_wrong = ones_wrong + 1
                
        if tie:
            total_ties = total_ties + 1
            if expected == "0":
                zeros_total_ties = zeros_total_ties + 1
            elif expected == "1":
                ones_total_ties = ones_total_ties + 1
                
    avg_time = float(time.time() - avg_time) / float(len(test_data_combined)) 
    
    print("Number of observations: ", len(test_data_combined))
    print("Predicted correctly: ", correct_predictions)
    print("Predicted wrongly: ", wrong_predictions)
    print("Predicted zeros: ", zeros_predicted)
    print("Predicted ones: ", ones_predicted)
    print("Zeros correct: ", zeros_correct)
    print("Ones correct: ", ones_correct)
    print("Zeros wrong: ", zeros_wrong)
    print("Ones Wrong: ", ones_wrong)
    print("Total ties: ", total_ties)
    print("Zeros total ties: ", zeros_total_ties)
    print("Ones total ties: ", ones_total_ties)
    print("Avg. Time: ", avg_time, " seconds.")
    return correct_predictions, [
        len(test_data_combined), correct_predictions, wrong_predictions, zeros_predicted, ones_predicted,
        zeros_correct, ones_correct, zeros_wrong, ones_wrong, total_ties, zeros_total_ties, ones_total_ties,
        avg_time
    ]

#Evaluates Firminos's wisard implementation
def evaluate_performance2(w, test_data_combined):
    #print("Number of observations: ", test_data_combined)
    correct_predictions = 0
    wrong_predictions = 0
    zeros_predicted = 0
    ones_predicted = 0
    zeros_correct = 0
    ones_correct = 0
    zeros_wrong = 0
    ones_wrong = 0
    avg_time = time.time()
    
    for combined in test_data_combined:
        prediction = w.predict([combined[0]])
        if prediction[0] == "0":
            #print("Prediction: ", prediction[0], combined)
            zeros_predicted = zeros_predicted + 1
        elif prediction[0] == "1":
            ones_predicted = ones_predicted + 1
        #print(prediction)
        expected = combined[1]
        #print(prediction, expected)
        if prediction[0] == expected:
            #print("Correct!")
            correct_predictions = correct_predictions + 1
            
            if prediction[0] == "0":
                zeros_correct = zeros_correct + 1
            elif prediction[0] == "1":
                ones_correct = ones_correct + 1
        else:
            wrong_predictions = wrong_predictions + 1
            
            if prediction[0] == "0":
                zeros_wrong = zeros_wrong + 1
            elif prediction[0] == "1":
                ones_wrong = ones_wrong + 1
                
    avg_time = float(time.time() - avg_time) / float(len(test_data_combined)) 
    
    print("Number of observations: ", len(test_data_combined))
    print("Predicted correctly: ", correct_predictions)
    print("Predicted wrongly: ", wrong_predictions)
    print("Predicted zeros: ", zeros_predicted)
    print("Predicted ones: ", ones_predicted)
    print("Zeros correct: ", zeros_correct)
    print("Ones correct: ", ones_correct)
    print("Zeros wrong: ", zeros_wrong)
    print("Ones Wrong: ", ones_wrong)
    print("Avg. Time: ", avg_time, " seconds.")
    return correct_predictions, [
        len(test_data_combined), correct_predictions, wrong_predictions, zeros_predicted, ones_predicted,
        zeros_correct, ones_correct, zeros_wrong, ones_wrong, avg_time
    ]

In [7]:
def write_result_line(algorithm, seed, encoding_type, category_file, field_size, training_set_distribuition, a_tuple_size, bleaching,
                     training_time, in_sample_additional_info, in_sample_evaluation_time, out_sample_additional_info,
                     out_sample_evaluation_time, in_sample_performance, training_combined, training_set, out_sample_performance,
                     test_combined, test_set, output_file):
    line = ""
    line_contents = [
        # Algorithm
        str(algorithm) + ";",
        # Seed
        str(seed) + ";",
        # Encoding type
        str(encoding_type) + ";",
        # Category file's name
        str(category_file) + ";",
        # Field size
        str(field_size) + ";",
        # training / test
        str(training_set_distribuition[0]) + "/" + str(training_set_distribuition[1]) + ";",
        # tuple size
        str(a_tuple_size) + ";",
        # bleaching active or not
        str(bleaching) + ";",
        # training time in seconds
        str(training_time) + ";",
        # in sample pattern average evaluation time in seconds
        str(in_sample_additional_info[12]) + ";",
        # in sample total evaluation time in seconds
        str(in_sample_evaluation_time) + ";",
        # out sample pattern average evaluation time in seconds
        str(out_sample_additional_info[12]) + ";",
        # out sample total evaluation time in seconds
        str(out_sample_evaluation_time) + ";",

        # total traning observations
        str(training_set_distribuition[0] + training_set_distribuition[1]) + ";",
        # total of correct prediction in the training dataset
        str(in_sample_performance) + ";",
        # percentage of right answers
        str(float(in_sample_performance) / float(len(training_combined))) + ";",
        # total approved in the training dataset
        str(training_set["project_is_approved"].sum()) + ";",
        # total approved correctly predicted in the training dataset
        str(in_sample_additional_info[6]) + ";",
        # total approved wrongly predicted in the training dataset
        str(in_sample_additional_info[8]) + ";",
        # percentage of approved projects predicted correctly in the training dataset
        str(float(in_sample_additional_info[6]) / float(training_set["project_is_approved"].sum())) + ";",
        # total reproved in the training dataset
        str((training_set["project_is_approved"].sum() - len(training_set["project_is_approved"])) * -1) + ";",
        # total reproved correctly predicted in the training dataset
        str(in_sample_additional_info[5]) + ";",
        # total reproved wrongly predicted in the training dataset
        str(in_sample_additional_info[7]) + ";",
        # percentage of reproved projects predicted correctly in the training dataset
        str(float(in_sample_additional_info[5]) / float((training_set["project_is_approved"].sum() - len(training_set["project_is_approved"])) * -1)) + ";",


        # total test observations
        str(len(test_set["project_is_approved"])) + ";",
        # total of correct prediction in the test dataset
        str(out_sample_performance) + ";",
        # percentage of right answers
        str(float(out_sample_performance) / float(len(test_combined))) + ";",
        # total approved in the test dataset
        str(test_set["project_is_approved"].sum()) + ";",
        # total approved correctly predicted in the test dataset
        str(out_sample_additional_info[6]) + ";",
        # total approved wrongly predicted in the test dataset
        str(out_sample_additional_info[8]) + ";",
        # percentage of approved projects predicted correctly in the test dataset
        str(float(out_sample_additional_info[6]) / float(test_set["project_is_approved"].sum())) + ";",
        # total reproved in the test dataset
        str((test_set["project_is_approved"].sum() - len(test_set["project_is_approved"])) * -1) + ";",
        # total reproved correctly predicted in the training dataset
        str(out_sample_additional_info[5]) + ";",
        # total reproved wrongly predicted in the training dataset
        str(out_sample_additional_info[7]) + ";",
        # percentage of reproved projects predicted correctly in the training dataset
        str(float(out_sample_additional_info[5]) / float((test_set["project_is_approved"].sum() - len(test_set["project_is_approved"])) * -1)) + ";",

        # total ties
        str(out_sample_additional_info[9]) + ";",
        # total ties when prediction should have been zero
        str(out_sample_additional_info[10]) + ";",
        # total ties when prediction should have been one
        str(out_sample_additional_info[11]) + ";",

        "\n",
        #str() + ";",
    ]

    for content in line_contents:
        #print(content)
        line = line + content

    print(line)

    file = open(output_file, "a+")
    file.write(line)
    file.close()

In [8]:
def experiment(params): #training_set_distribuitions, tuple_sizes, bleaching_mode = [False]):
    output_file = "insights/results_experiment" + datetime.now().strftime('%Y%m%d%H%M%S') + ".csv"
    file = open(output_file, "w")
    file.write("algorithm;seed;encoding;category_file;field_size;data_distribution;tuple_size;bleaching_active;total_training_time;avg_in_sample_evaluation_time;total_in_sample_evaluation_time;avg_out_sample_evaluation_time;total_out_sample_evaluation_time;total_training_data;total_correct_training;" +
               "percent_correct_training;total_approved_training;correctly_approved_training;wrongly_approved_training;" +
               "percent_approved_correctly_training;total_reproved_training;correctly_reproved_training;" +
               "wrongly_reproved_training;percent_reproved_correctly_training;total_test_data;total_correct_test;" +
               "percent_correct_test;total_approved_test;correctly_approved_test;wrongly_approved_test;" +
               "percent_approved_correctly_test;total_reproved_test;correctly_reproved_test;" +
               "wrongly_reproved_test;percent_reproved_correctly_test;total_ties;ties_for_zeros;ties_for_ones\n"
              )
    file.close()
    
    for seed in params["seeds"]:
        print("Seed: ", seed)
        
        for encoding_type in params["encoding_types"]:
            print("Input encoding type: ", encoding_type)
            
            for field_size in params["field_sizes"]:
                print("Input field sizes: " + str(field_size) if field_size != 0 else "Default")
                
                for category_file_name in params["category_files"][field_size]:
                    print("Category file: " + category_file_name)
                    file_ = open(category_file_name, "rb")
                    category_text_data = pickle.load(file_)
                    
                    for training_set_distribuition in params["training_set_distribuitions"]:
                        print("\nTraining with a training set distribution of ", 
                              training_set_distribuition[0], training_set_distribuition[1],
                              " for approved and repproved, respectively.\n")


                        current_params = {
                            #"algorithm": algorithm,
                            "seed": seed, # 0 = default
                            "encoding": encoding_type,
                            "encoding_threshold": None,
                            "field_size": field_size, # 0 = default
                            "training_set_distribuition": training_set_distribuition,
                            "tuple_size": None,
                            "bleaching_mode": None,
                            "category_file_name": category_file_name,
                            "category_text_data": category_text_data,

                            # TODO change the code to iterate over the possible values of the following parameters
                            "min_score": params["min_score"],
                            "threshold": params["threshold"], 
                            "discriminator_limit": params["discriminator_limit"] 
                        }

                        training_input, expected_output, training_combined, test_input, test_expected_output, test_combined, training_set, test_set = getData(training_set_distribuition[0], training_set_distribuition[1], current_params)

                        for a_tuple_size in params["tuple_sizes"]:
                            print("Training with a tupple of size: ", a_tuple_size)

                            for bleaching in params["bleaching_modes"]:
                                print("Bleaching is set to: ", bleaching, "\n")

                                for algorithm in params["algorithms"]:
                                    current_params["algorithm"] = algorithm

                                    training_time = time.time()
                                    wann = train(algorithm, training_input, expected_output, current_params, a_tuple_size, bleaching, seed)
                                    training_time = time.time() - training_time

                                    print("Training complete. " + str(training_time) + " seconds.")

                                    in_sample_evaluation_time = time.time()
                                    in_sample_performance, in_sample_additional_info =  evaluate_performance(wann, training_combined, current_params)
                                    in_sample_evaluation_time = time.time() - in_sample_evaluation_time

                                    # Evaluates Guilherme's wisard implementation
                                    print("In-sample performance: ", float(in_sample_performance) / float(len(training_combined)))
                                    print("Ones distribution: ", float(training_set["project_is_approved"].sum()) / float(len(training_set["project_is_approved"])))
                                    print("Ones: ", training_set["project_is_approved"].sum(), "Zeros: ", training_set["project_is_approved"].sum() - len(training_set["project_is_approved"]))
                                    print("\n")

                                    out_sample_evaluation_time = time.time()
                                    out_sample_performance, out_sample_additional_info =  evaluate_performance(wann, test_combined, current_params)
                                    out_sample_evaluation_time = time.time() - out_sample_evaluation_time

                                    print("Expected out-sample performance: ", float(out_sample_performance) / float(len(test_combined)))
                                    print("Ones distribution: ", float(test_set["project_is_approved"].sum()) / float(len(test_set["project_is_approved"])))
                                    print("Ones: ", test_set["project_is_approved"].sum(), "Zeros: ", (test_set["project_is_approved"].sum() - len(test_set["project_is_approved"])), "\n\n")



                                    write_result_line(algorithm, seed, encoding_type, category_file_name, field_size, training_set_distribuition, a_tuple_size, bleaching,
                                                     training_time, in_sample_additional_info, in_sample_evaluation_time, out_sample_additional_info,
                                                     out_sample_evaluation_time, in_sample_performance, training_combined, training_set, out_sample_performance,
                                                     test_combined, test_set, output_file)


    

In [ ]:
tuple_sizes = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 50, 100]
#tuple_sizes = [2]
#tuple_sizes = [2, 3, 4, 5, 7, 8, 10]
#training_set_distribuitions = [[5, 5], [10, 10], [20, 20], [30, 30], [50, 50], [75, 75], [86, 14], [100, 100]]
training_set_distribuitions = [[5, 5], [10, 10], [100, 100]]#, [1000, 1000]]

param_variations = {
    "seeds": [0], # 0 = default
    #"encoding_types": ["standard", "thermometer", "onehot"],
    "encoding_types": ["thermometer"],
    "field_sizes": [52, 104, 208], # 0 = default
    "training_set_distribuitions": training_set_distribuitions,
    "tuple_sizes": tuple_sizes,
    #"bleaching_modes": [False, True],
    "bleaching_modes": [True],
    #"algorithms": ["wisard","cluswisard"],
    "algorithms": ["wisard"],
    #"algorithms": ["wisard"],
    
    
    "category_files": {
        52: [
            "compact_Applied_Learning_50.pickle",
            "compact_Health_Sports_50.pickle",
            "compact_History_Civics_50.pickle",
            "compact_Literacy_Language_50.pickle",
            "compact_Math_Science_50.pickle",
            "compact_Music_The_Arts_50.pickle",
            "compact_Special_Needs_50.pickle",
            "compact_Warmth_Care_Hunger_50.pickle",
        ],
        104: [
            "compact_Applied_Learning_100.pickle",
            "compact_Health_Sports_100.pickle",
            "compact_History_Civics_100.pickle",
            "compact_Literacy_Language_100.pickle",
            "compact_Math_Science_100.pickle",
            "compact_Music_The_Arts_100.pickle",
            "compact_Special_Needs_100.pickle",
            "compact_Warmth_Care_Hunger_100.pickle",
        ],
        208: [
            "compact_Applied_Learning_200.pickle",
            "compact_Health_Sports_200.pickle",
            "compact_History_Civics_200.pickle",
            "compact_Literacy_Language_200.pickle",
            "compact_Math_Science_200.pickle",
            "compact_Music_The_Arts_200.pickle",
            "compact_Special_Needs_200.pickle",
            "compact_Warmth_Care_Hunger_200.pickle",
        ]
    },
    
    "current_category_file": None,
    
    # cluswisard parameters
    "min_score": 0.1,
    "threshold": 10, 
    "discriminator_limit": 5
}


experiment(param_variations)

Seed:  0
Input encoding type:  thermometer
Input field sizes: 52
Category file: compact_Applied_Learning_50.pickle

Training with a training set distribution of  5 5  for approved and repproved, respectively.

train_data shape (16811, 67)
Total data:  16811
Total aproved:  13827
Total reproved:  2984
Percent aproved:  0.8224971744690976
Percent reproved:  0.17750282553090235 

Distribution over a random sample of 182080 observations used to get the observations to train the classifier:  0.8224971744690976
Total aproved in that sample:  13827 

Total training data:  10
Total aproved:  5
Total reproved:  5
Percent aproved:  0.5
Percent reproved:  0.5 

Total test data:  16801
Total aproved:  13822
Total reproved:  2979
Percent aproved:  0.8226891256472829
Percent reproved:  0.17731087435271708 

Training set + test set:  16811
Dim.: (10, 65)
Dim.: (16801, 65)
Converting dataframe of shape  (10, 65)  to a list of binary values.
1 thermometer 4 52 None
Number of bits in the input:  414
Con

Number of observations:  16801
Predicted correctly:  6624
Predicted wrongly:  10177
Predicted zeros:  11140
Predicted ones:  5661
Zeros correct:  1971
Ones correct:  4653
Zeros wrong:  9169
Ones Wrong:  1008
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00014855485467369815  seconds.
Expected out-sample performance:  0.3942622462948634
Ones distribution:  0.8226891256472829
Ones:  13822 Zeros:  -2979 


wisard;0;thermometer;compact_Applied_Learning_50.pickle;52;5/5;8;True;0.001360177993774414;0.00025882720947265623;0.003935813903808594;0.00014855485467369815;2.5061964988708496;10;10;1.0;5;5;0;1.0;5;5;0;1.0;16801;6624;0.3942622462948634;13822;4653;1008;0.3366372449717841;2979;1971;9169;0.6616314199395771;0;0;0;

Training with a tupple of size:  9
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.0017118453979492188 seconds.
Number of observations:  10
Predicted correctly:  10
Predicted wrongly:  0
Predicted zeros:  5
Predicted ones:  5
Zeros co

Number of observations:  16801
Predicted correctly:  7640
Predicted wrongly:  9161
Predicted zeros:  10056
Predicted ones:  6745
Zeros correct:  1937
Ones correct:  5703
Zeros wrong:  8119
Ones Wrong:  1042
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00011311162902403358  seconds.
Expected out-sample performance:  0.4547348372120707
Ones distribution:  0.8226891256472829
Ones:  13822 Zeros:  -2979 


wisard;0;thermometer;compact_Applied_Learning_50.pickle;52;5/5;15;True;0.0018167495727539062;0.00015871524810791017;0.0026433467864990234;0.00011311162902403358;1.9030239582061768;10;10;1.0;5;5;0;1.0;5;5;0;1.0;16801;7640;0.4547348372120707;13822;5703;1042;0.4126030965128057;2979;1937;8119;0.6502181940248406;0;0;0;

Training with a tupple of size:  16
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.0010714530944824219 seconds.
Number of observations:  10
Predicted correctly:  10
Predicted wrongly:  0
Predicted zeros:  5
Predicted ones:  5
Zeros

Number of observations:  16801
Predicted correctly:  11998
Predicted wrongly:  4803
Predicted zeros:  2970
Predicted ones:  13831
Zeros correct:  573
Ones correct:  11425
Zeros wrong:  2397
Ones Wrong:  2406
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00011361893369544015  seconds.
Expected out-sample performance:  0.7141241592762335
Ones distribution:  0.8226891256472829
Ones:  13822 Zeros:  -2979 


wisard;0;thermometer;compact_Applied_Learning_50.pickle;52;5/5;30;True;0.0009424686431884766;9.162425994873047e-05;0.008998394012451172;0.00011361893369544015;1.9102771282196045;10;10;1.0;5;5;0;1.0;5;5;0;1.0;16801;11998;0.7141241592762335;13822;11425;2406;0.8265808131963537;2979;573;2397;0.19234642497482377;0;0;0;

Training with a tupple of size:  50
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.0012938976287841797 seconds.
Number of observations:  10
Predicted correctly:  10
Predicted wrongly:  0
Predicted zeros:  5
Predicted ones:  5
Zero

Number of observations:  16791
Predicted correctly:  10092
Predicted wrongly:  6699
Predicted zeros:  6375
Predicted ones:  10416
Zeros correct:  1325
Ones correct:  8767
Zeros wrong:  5050
Ones Wrong:  1649
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00013112428324494195  seconds.
Expected out-sample performance:  0.6010362694300518
Ones distribution:  0.822881305461259
Ones:  13817 Zeros:  -2974 


wisard;0;thermometer;compact_Applied_Learning_50.pickle;52;10/10;5;True;0.0025031566619873047;0.00016267299652099608;0.0047876834869384766;0.00013112428324494195;2.2038962841033936;20;20;1.0;10;10;0;1.0;10;10;0;1.0;16791;10092;0.6010362694300518;13817;8767;1649;0.6345082145183469;2974;1325;5050;0.44552790854068597;0;0;0;

Training with a tupple of size:  6
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.004596710205078125 seconds.
Number of observations:  20
Predicted correctly:  20
Predicted wrongly:  0
Predicted zeros:  10
Predicted ones:  1

...

Number of observations:  6905
Predicted correctly:  3769
Predicted wrongly:  3136
Predicted zeros:  3101
Predicted ones:  3804
Zeros correct:  501
Ones correct:  3268
Zeros wrong:  2600
Ones Wrong:  536
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00018627510650877846  seconds.
Expected out-sample performance:  0.5458363504706735
Ones distribution:  0.8498189717595945
Ones:  5868 Zeros:  -1037 


wisard;0;thermometer;compact_History_Civics_100.pickle;104;10/10;15;True;0.0036089420318603516;0.00016458034515380858;0.013167381286621094;0.00018627510650877846;1.2973988056182861;20;20;1.0;10;10;0;1.0;10;10;0;1.0;6905;3769;0.5458363504706735;5868;3268;536;0.5569188820722563;1037;501;2600;0.4831243972999036;0;0;0;

Training with a tupple of size:  16
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.003355741500854492 seconds.
Number of observations:  20
Predicted correctly:  20
Predicted wrongly:  0
Predicted zeros:  10
Predicted ones:  10
Zeros co

Number of observations:  6905
Predicted correctly:  4590
Predicted wrongly:  2315
Predicted zeros:  1926
Predicted ones:  4979
Zeros correct:  324
Ones correct:  4266
Zeros wrong:  1602
Ones Wrong:  713
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00016603138376024648  seconds.
Expected out-sample performance:  0.664735698769008
Ones distribution:  0.8498189717595945
Ones:  5868 Zeros:  -1037 


wisard;0;thermometer;compact_History_Civics_100.pickle;104;10/10;30;True;0.003123760223388672;0.00016007423400878906;0.01131129264831543;0.00016603138376024648;1.156229019165039;20;20;1.0;10;10;0;1.0;10;10;0;1.0;6905;4590;0.664735698769008;5868;4266;713;0.7269938650306749;1037;324;1602;0.3124397299903568;0;0;0;

Training with a tupple of size:  50
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.0029251575469970703 seconds.
Number of observations:  20
Predicted correctly:  20
Predicted wrongly:  0
Predicted zeros:  10
Predicted ones:  10
Zeros correc

Number of observations:  6725
Predicted correctly:  4673
Predicted wrongly:  2052
Predicted zeros:  1819
Predicted ones:  4906
Zeros correct:  357
Ones correct:  4316
Zeros wrong:  1462
Ones Wrong:  590
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.0002570323518660875  seconds.
Expected out-sample performance:  0.6948698884758364
Ones distribution:  0.859182156133829
Ones:  5778 Zeros:  -947 


wisard;0;thermometer;compact_History_Civics_100.pickle;104;100/100;5;True;0.02626633644104004;0.00031343817710876464;0.0709998607635498;0.0002570323518660875;1.7376956939697266;200;186;0.93;100;98;12;0.98;100;88;2;0.88;6725;4673;0.6948698884758364;5778;4316;590;0.7469712703357563;947;357;1462;0.37697993664202745;0;0;0;

Training with a tupple of size:  6
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.03228354454040527 seconds.
Number of observations:  200
Predicted correctly:  190
Predicted wrongly:  10
Predicted zeros:  96
Predicted ones:  104
Zeros

Number of observations:  6725
Predicted correctly:  4065
Predicted wrongly:  2660
Predicted zeros:  2513
Predicted ones:  4212
Zeros correct:  400
Ones correct:  3665
Zeros wrong:  2113
Ones Wrong:  547
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00019950076106755706  seconds.
Expected out-sample performance:  0.6044609665427509
Ones distribution:  0.859182156133829
Ones:  5778 Zeros:  -947 


wisard;0;thermometer;compact_History_Civics_100.pickle;104;100/100;12;True;0.02624034881591797;0.00023736953735351563;0.051013946533203125;0.00019950076106755706;1.351288080215454;200;200;1.0;100;100;0;1.0;100;100;0;1.0;6725;4065;0.6044609665427509;5778;3665;547;0.6343025268258913;947;400;2113;0.42238648363252373;0;0;0;

Training with a tupple of size:  13
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.02750372886657715 seconds.
Number of observations:  200
Predicted correctly:  200
Predicted wrongly:  0
Predicted zeros:  100
Predicted ones:  100
Ze

Number of observations:  6725
Predicted correctly:  3797
Predicted wrongly:  2928
Predicted zeros:  2845
Predicted ones:  3880
Zeros correct:  432
Ones correct:  3365
Zeros wrong:  2413
Ones Wrong:  515
Total ties:  0
Zeros total ties:  0
Ones total ties:  0
Avg. Time:  0.00019009242713672965  seconds.
Expected out-sample performance:  0.5646096654275093
Ones distribution:  0.859182156133829
Ones:  5778 Zeros:  -947 


wisard;0;thermometer;compact_History_Civics_100.pickle;104;100/100;19;True;0.03264451026916504;0.0001853513717651367;0.0456085205078125;0.00019009242713672965;1.284433126449585;200;200;1.0;100;100;0;1.0;100;100;0;1.0;6725;3797;0.5646096654275093;5778;3365;515;0.5823814468674282;947;432;2413;0.45617740232312565;0;0;0;

Training with a tupple of size:  20
Bleaching is set to:  True 

Algorithm: WiSARD.
Training complete. 0.04998183250427246 seconds.
Number of observations:  200
Predicted correctly:  200
Predicted wrongly:  0
Predicted zeros:  100
Predicted ones:  100
Zeros